In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import librosa

In [2]:
# EXTRACT_DATASET = True
# DATASET_URL = "https://goo.gl/8hY5ER"

# # set paths to the UrbanSound8K dataset and metadata file
# US8K_AUDIO_PATH = os.path.abspath('UrbanSound8K/audio/')
# US8K_METADATA_PATH = os.path.abspath('UrbanSound8K/metadata/UrbanSound8K.csv')

US8K_AUDIO_PATH = os.path.abspath('F:/DATAS/UrbanSound8K/UrbanSound8K/audio/')
US8K_METADATA_PATH = os.path.abspath('F:/DATAS/UrbanSound8K/UrbanSound8K/metadata/UrbanSound8K.csv')

# load the csv metadata file into a Pandas DataFrame structure
us8k_metadata_df = pd.read_csv(US8K_METADATA_PATH,
                               usecols=["slice_file_name", "fold", "classID"],
                               dtype={"fold": "uint8", "classID" : "uint8"})
us8k_metadata_df

,slice_file_name,fold,classID
0,100032-3-0-0.wav,5,3
1,100263-2-0-117.wav,5,2
2,100263-2-0-121.wav,5,2
3,100263-2-0-126.wav,5,2
4,100263-2-0-137.wav,5,2
...,...,...,...
8727,99812-1-2-0.wav,7,1
8728,99812-1-3-0.wav,7,1
8729,99812-1-4-0.wav,7,1
8730,99812-1-5-0.wav,7,1


In [3]:
HOP_LENGTH = 512        # number of samples between successive frames
WINDOW_LENGTH = 512     # length of the window in samples
N_MEL = 128             # number of Mel bands to generate

def compute_melspectrogram_with_fixed_length(audio, sampling_rate, num_of_samples=128):
    try:
        # compute a mel-scaled spectrogram
        melspectrogram = librosa.feature.melspectrogram(y=audio, 
                                                        sr=sampling_rate, 
                                                        hop_length=HOP_LENGTH,
                                                        win_length=WINDOW_LENGTH, 
                                                        n_mels=N_MEL)

        # convert a power spectrogram to decibel units (log-mel spectrogram)
        melspectrogram_db = librosa.power_to_db(melspectrogram, ref=np.max)
        
        melspectrogram_length = melspectrogram_db.shape[1]
        
        # pad or fix the length of spectrogram 
        if melspectrogram_length != num_of_samples:
            melspectrogram_db = librosa.util.fix_length(melspectrogram_db, 
                                                        size=num_of_samples, 
                                                        axis=1, 
                                                        constant_values=(0, -80.0))
    except Exception as e:
        print("\nError encountered while parsing files\n>>", e)
        return None 
    
    return melspectrogram_db

In [4]:
SOUND_DURATION = 2.95   # fixed duration of an audio excerpt in seconds

features = []

# iterate through all dataset examples and compute log-mel spectrograms
for index, row in tqdm(us8k_metadata_df.iterrows(), total=len(us8k_metadata_df)):
    file_path = f'{US8K_AUDIO_PATH}/fold{row["fold"]}/{row["slice_file_name"]}'
    audio, sample_rate = librosa.load(file_path, duration=SOUND_DURATION, res_type='kaiser_fast')
    
    melspectrogram = compute_melspectrogram_with_fixed_length(audio, sample_rate)
    label = row["classID"]
    fold = row["fold"]
    
    features.append([melspectrogram, label, fold])

# convert into a Pandas DataFrame 
us8k_df = pd.DataFrame(features, columns=["melspectrogram", "label", "fold"])

 41%|███████████████████████████████▋                                              | 3553/8732 [01:39<02:23, 36.07it/s]C:\Users\zhaoke\miniconda3\envs\torch-0\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
 95%|██████████████████████████████████████████████████████████████████████████▎   | 8322/8732 [03:46<00:08, 47.93it/s]C:\Users\zhaoke\miniconda3\envs\torch-0\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\zhaoke\miniconda3\envs\torch-0\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 8732/8732 [03:57<00:00, 36.84it/s]


In [7]:
for index, row in tqdm(us8k_df.iterrows(), total=len(us8k_metadata_df)):
    print(row["melspectrogram"].shape)
    break

  0%|                                                                                         | 0/8732 [00:00<?, ?it/s]

(128, 128)


In [5]:
# write the Pandas DataFrame object to .pkl file
WRITE_DATA = True

if WRITE_DATA:
  us8k_df.to_pickle("F:/DATAS/UrbanSound8K/us8k_df.pkl")